In [ ]:
"""\
# Copyright (C) 2024 Jesús Bautista Villar <jesbauti20@gmail.com>
"""
!python3 -V || python -V

In [ ]:
import numpy as np
import os
import sys
from tqdm import tqdm

# Graphic tool
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

# Configure matplotlib for paper figures
import matplotlib
matplotlib.rcParams["mathtext.fontset"] = "cm"
matplotlib.rc("font", **{"size" : 12})

# Animation tools
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.animation import PillowWriter

# --------------------------------------------------------------------------------------

# Swarm Systems Lab PySimUtils
from ssl_pysimutils import createDir, gen_Z_ring, uniform_distrib

# Python project to path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import the simulator
from cfc.simulator import simulator

# --------------------------------------------------------------------------------------

# Output directory
OUTPUT_PATH = os.path.join("..", "output")

createDir(OUTPUT_PATH)

In [12]:
p = np.array([[1,2],[1,4],[1,2]])
pcx, pcy = np.sum(p, axis=0)/3
pcx

1.0

In [ ]:
# Generate the robots distribution and set the initial positions
N = 10
lims = [1.5, 2]
rc0 = [0, 0]

p0 = uniform_distrib(N=N, lims=lims, rc0=rc0)

# Generate the graph
Z = gen_Z_ring()

# Generating the simulator class
dt = 0.1
lambda_d = [30, 5]

sim = simulator(p0=p0, Z=Z, dt=dt)

# ----------------------------------------------------------------------
# Execute the simulation given a desired final time stamp
# ----------------------------------------------------------------------
tf = 15

t_list = np.arange(0, tf, dt)
for it in tqdm(range(len(t_list))):
    sim.int_euler()

In [ ]:
t_data = np.array(sim.data["t"])
p_data = np.array(sim.data["p"])
e_data = np.array(sim.data["e"])


# ----------------------------------------------------------------------

fig = plt.figure()
ax = fig.subplots()

ax.plot(p_data[:,0,0], p_data[:,0,1])
ax.set_aspect("equal")
ax.grid(True)
plt.show()

# ----------------------------------------------------------------------

plt.plot(t_data, e_data)
plt.title(r"$e$")
plt.grid(True)
# plt.ylim([-1,1])
plt.show()